In [142]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [143]:
#loading the datasets
dataset1 = pd.read_csv("../data/raw/games.csv", index_col=False)
dataset2 = pd.read_csv("../data/raw/game_data_all.csv")


C:\Users\lotec\AppData\Local\Temp\ipykernel_21344\431428641.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  dataset1 = pd.read_csv("../data/raw/games.csv", index_col=False)


In [144]:
#Selecting only the columns we want
dataset1 = dataset1[[
    "AppID",
    "Name",
    "Release date",
    "Price",
    "Windows",
    "Mac",
    "Linux",
    "Metacritic score",
    "Categories",
    "Genres",
    "Tags"
]]

dataset2 = dataset2[[
    "game",
    "link",
    "peak_players",
    "positive_reviews",
    "negative_reviews",
    "total_reviews"
]]


In [145]:
#Preparing for merge

#correcting the 'link' in dataset2 to only be an int. Ex: "/app/2231450/" -> "2231450"
dataset2["app_id"] = dataset2["link"].str.extract(r"/app/(\d+)/")
dataset2["app_id"] = dataset2["app_id"].astype(int)
dataset2 = dataset2.drop(columns=["link"])

#renaming appId to be the same in dataset2
dataset1 = dataset1.rename(columns={"AppID": "app_id"})
dataset1["app_id"] = dataset1["app_id"].astype(int)



In [147]:
merged_df = pd.merge(dataset1, dataset2, on="app_id", how="inner")
print(merged_df)

        app_id                             Name  Release date  Price  \
0        20200                 Galactic Bowling  Oct 21, 2008  19.99   
1       655370                     Train Bandit  Oct 12, 2017   0.99   
2      1355720                         Henosis™  Jul 23, 2020   5.99   
3      1139950            Two Weeks in Painland   Feb 3, 2020   0.00   
4      1469160                   Wartune Reborn  Feb 26, 2021   0.00   
...        ...                              ...           ...    ...   
66964  2009390           Dream Three Kingdoms 2  Aug 31, 2022   0.00   
66965  1530300                       Auto Chess   Dec 7, 2022   0.00   
66966   528940  Birthright Cataclysm - Overture   Dec 9, 2020   0.00   
66967  1734500              Knights and Castles  Oct 19, 2021   2.99   
66968  1509950                      Empire IDLE  Jan 16, 2021   1.99   

                               Windows   Mac  Linux  Metacritic score  \
0                                  NaN  True  False           